### I was unsuccessful in sending a working online prediction request to the endpoint in the project timeframe.  
Work toward this end is recorded below.

In [1]:
from google.cloud import bigquery
from google.cloud import aiplatform as vertexai
import numpy as np
import pandas as pd
from google.cloud.bigquery import Client, QueryJobConfig
client = Client()

In [2]:
PROJECT_ID = 'york-bb-cohort'
BQ_LOCATION = 'US'
REGION = 'us-east1'
GCS_BUCKET = f"{PROJECT_ID}-akane_bucket"
# Bucket created via console
BQ_DATASET = f"{PROJECT_ID}:akane_bqds"

### Functions to transform categorical fields

List of all categorical features in the order in which they appear in the query

In [ ]:
CATEGORICAL_FEATURES = ['sku',
                        'networkDomain',
                        'country',
                        'operatingSystem',
                        'browser',
                        'source',
                        'channelGrouping',
                        'isEntrance',
                        'isExit',
                        'type',
                        'socialNetwork',
                        'hasSocialSourceReferral',
                        'contentGroup1',
                        'contentGroup2',
                        'contentGroup3',
                        'previousContentGroup1',
                        'previousContentGroup2',
                        'previousContentGroup3',
                        'previousContentGroup4',
                        'previousContentGroup5',
                        'campaign',
                        'medium',
                        'isMobile',
                        'deviceCategory'
                       ]

### Get endpoint where model is deployed

In [3]:
endpoint = vertexai.Endpoint(endpoint_name = 'projects/319407066227/locations/us-central1/endpoints/6657263630230224896')

### Grab a couple of entries for requesting prediction - from outside the training set

In [24]:
%%bigquery sample_df --project $PROJECT_ID

SELECT
* EXCEPT(hitId)
FROM `york-bb-cohort.akane_bqds.some_test`
WHERE hitId = '149231360969731406445272116662' OR hitId = '149231360969731406445272116664'

Query is running:   0%|          |

Downloading:   0%|          |

In [25]:
sample_df

,visitNumber,sku,label,visitStartTime,date,networkDomain,country,operatingSystem,browser,source,...,previousContentGroup4,previousContentGroup5,campaign,medium,hits,pageviews,timeOnSite,newVisits,isMobile,deviceCategory
0,1,none_or_many,0,1492313609,20170415,other,United States,Macintosh,Chrome,google,...,(not set),(not set),AW - Accessories,cpc,13,11,504,1,False,desktop
1,1,GGOEGAAX0031,1,1492313609,20170415,other,United States,Macintosh,Chrome,google,...,(not set),(not set),AW - Accessories,cpc,13,11,504,1,False,desktop


In [8]:
import json

In [26]:
sample1 = dict(sample_df.iloc[0])
sample1

{'visitNumber': 1,
 'sku': 'none_or_many',
 'label': 0,
 'visitStartTime': 1492313609,
 'date': 20170415,
 'networkDomain': 'other',
 'country': 'United States',
 'operatingSystem': 'Macintosh',
 'browser': 'Chrome',
 'source': 'google',
 'channelGrouping': 'Paid Search',
 'hitNumber': 2,
 'time': 16628,
 'isEntrance': <NA>,
 'isExit': <NA>,
 'type': 'PAGE',
 'step': 1,
 'socialNetwork': '(not set)',
 'hasSocialSourceReferral': 'No',
 'contentGroup1': '(not set)',
 'contentGroup2': 'Apparel',
 'contentGroup3': '(not set)',
 'previousContentGroup1': '(not set)',
 'previousContentGroup2': '(not set)',
 'previousContentGroup3': '(not set)',
 'previousContentGroup4': '(not set)',
 'previousContentGroup5': '(not set)',
 'campaign': 'AW - Accessories',
 'medium': 'cpc',
 'hits': 13,
 'pageviews': 11,
 'timeOnSite': 504,
 'newVisits': 1,
 'isMobile': False,
 'deviceCategory': 'desktop'}

In [17]:
# sample_df_requests = json.loads(sample_df.to_json(orient="records"))

In [27]:
prediction = endpoint.predict([sample1])
print(prediction.predictions)

ValueError: Unable to coerce value: 1